In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


from collections import defaultdict

from tqdm import tqdm
tqdm.pandas()

import numpy as np

Mounted at /content/drive


In [2]:
news_claim = pd.read_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim_NER.h5',compression='xz')
claim_propagation = pd.read_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation_NER.h5',compression='xz')
origin_data = pd.read_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/origin_data_NER.h5',compression='xz')

In [3]:
news_claim

,text,source,claim_num,group,NER
0,the (covid 19) cases are going up but it's bec...,politifact,0,1,"[(19, CARDINAL)]"
1,every election year has a disease sars in 200...,politifact,1,1,"[(every election year, DATE), (2004, DATE), (2..."
2,obama ordered cia to train isis jihadists,politifact,2,3,"[(obama, PERSON), (cia, ORG)]"
3,"says george soros said, im going to bring down...",politifact,3,3,"[(george soros, PERSON), (the united states, G..."
4,if you paid rent in march or april they got to...,politifact,4,1,"[(march or, DATE), (april, DATE)]"
...,...,...,...,...,...
1348,says a photo shows patrick mahomes wearing a s...,politifact,1348,3,"[(patrick mahomes, PERSON), (kansas, GPE)]"
1349,nancy pelosi may have just committed a violati...,politifact,1349,3,"[(nancy, PERSON), (18, CARDINAL), (2071, DATE)..."
1350,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"[(donald trump, PERSON), (one, CARDINAL), (dnc..."
1351,says eight iowa counties have more adults regi...,politifact,1351,3,"[(eight, CARDINAL), (iowa, GPE)]"


In [4]:
news_claim = news_claim.explode('NER')
claim_propagation = claim_propagation.explode('NER')
origin_data = origin_data.explode('NER')

news_claim['target_type'] = news_claim['NER'].apply(lambda x: x[1] if isinstance(x, tuple) else '')
news_claim['target'] = news_claim['NER'].apply(lambda x: x[0] if isinstance(x, tuple) else [])

claim_propagation['target_type'] = claim_propagation['NER'].apply(lambda x: x[1] if isinstance(x, tuple) else '')
claim_propagation['target'] = claim_propagation['NER'].apply(lambda x: x[0] if isinstance(x, tuple) else [])
#claim_propagation['target_type'] = claim_propagation['target_type'].replace(pd.NA, '')

origin_data['target_type'] = origin_data['NER'].apply(lambda x: x[1] if isinstance(x, tuple) else '')
origin_data['target'] = origin_data['NER'].apply(lambda x: x[0] if isinstance(x, tuple) else [])


In [5]:
news_claim

,text,source,claim_num,group,NER,target_type,target
0,the (covid 19) cases are going up but it's bec...,politifact,0,1,"(19, CARDINAL)",CARDINAL,19
1,every election year has a disease sars in 200...,politifact,1,1,"(every election year, DATE)",DATE,every election year
1,every election year has a disease sars in 200...,politifact,1,1,"(2004, DATE)",DATE,2004
1,every election year has a disease sars in 200...,politifact,1,1,"(2008, DATE)",DATE,2008
1,every election year has a disease sars in 200...,politifact,1,1,"(2010, DATE)",DATE,2010
...,...,...,...,...,...,...,...
1350,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"(dnc, ORG)",ORG,dnc
1351,says eight iowa counties have more adults regi...,politifact,1351,3,"(eight, CARDINAL)",CARDINAL,eight
1351,says eight iowa counties have more adults regi...,politifact,1351,3,"(iowa, GPE)",GPE,iowa
1352,says kobe bryants wife recorded her suicide af...,politifact,1352,3,"(kobe bryants, PERSON)",PERSON,kobe bryants


In [6]:
target_types_to_retain = ['PERSON', 'ORG', 'GPE','']

# Applying the transformation to the 'target' column
news_claim['target_filter'] = news_claim.target_type.isin(target_types_to_retain)
claim_propagation['target_filter'] = claim_propagation.target_type.isin(target_types_to_retain)
origin_data['target_filter'] = origin_data.target_type.isin(target_types_to_retain)

In [7]:
news_claim.target_type.value_counts()

PERSON         629
GPE            529
DATE           475
CARDINAL       429
ORG            276
NORP           185
               175
MONEY           78
ORDINAL         34
PERCENT         28
TIME            20
QUANTITY        12
WORK_OF_ART     11
LOC             10
FAC              7
PRODUCT          6
EVENT            6
Name: target_type, dtype: int64

In [8]:
news_claim[news_claim['target_filter']==True].target_type.value_counts()

PERSON    629
GPE       529
ORG       276
          175
Name: target_type, dtype: int64

In [9]:
news_claim.source.value_counts()

politifact    2114
snopes         796
Name: source, dtype: int64

In [11]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
news_claim[news_claim['target_filter']==True].to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim_NER_explode.h5',compression='xz')
news_claim[news_claim['target_filter']==True]

,text,source,claim_num,group,NER,target_type,target,target_filter
1,every election year has a disease sars in 200...,politifact,1,1,"(zika, PERSON)",PERSON,zika,True
2,obama ordered cia to train isis jihadists,politifact,2,3,"(obama, PERSON)",PERSON,obama,True
2,obama ordered cia to train isis jihadists,politifact,2,3,"(cia, ORG)",ORG,cia,True
3,"says george soros said, im going to bring down...",politifact,3,3,"(george soros, PERSON)",PERSON,george soros,True
3,"says george soros said, im going to bring down...",politifact,3,3,"(the united states, GPE)",GPE,the united states,True
...,...,...,...,...,...,...,...,...
1349,nancy pelosi may have just committed a violati...,politifact,1349,3,"(trumps, PERSON)",PERSON,trumps,True
1350,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"(donald trump, PERSON)",PERSON,donald trump,True
1350,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"(dnc, ORG)",ORG,dnc,True
1351,says eight iowa counties have more adults regi...,politifact,1351,3,"(iowa, GPE)",GPE,iowa,True


In [ ]:
origin_data[origin_data['target_filter']==True].to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/origin_data_NER_explode.h5',compression='xz')
origin_data[origin_data['target_filter']==True]

,create_date,tweet_id,similarity,like_count,retweet_count,post_text,claim_number,group,hashtag,tweet_user,NER,target_type,target,target_filter
0,2020-07-28T14:04:36.000Z,1288113129889517578,0.000000,2823,1050,The latest round of testing from the Miami Mar...,0,1,NaN,85952,"(the Miami Marlins, ORG)",ORG,the Miami Marlins,True
0,2020-07-28T14:04:36.000Z,1288113129889517578,0.000000,2823,1050,The latest round of testing from the Miami Mar...,0,1,NaN,85952,"(ESPN, ORG)",ORG,ESPN,True
2,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"(Hialeah, GPE)",GPE,Hialeah,True
2,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"(FL, GPE)",GPE,FL,True
2,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"(Carlos Hernandez, PERSON)",PERSON,Carlos Hernandez,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,2020-12-05T11:19:16.000Z,1335181945635004417,0.659882,1436,180,We broke the news on #OutLoudWithGiannoCaldwel...,298,1,OutLoudWithGiannoCaldwell,73973,"(Gianno Caldwell.https://t.co/aAKVBelRJO, PERSON)",PERSON,Gianno Caldwell.https://t.co/aAKVBelRJO,True
1772,2020-12-20T20:31:55.000Z,1340756841454850049,0.390273,171932,2654,I see your tweets and I’m trying to double and...,298,1,NaN,75482,"(MrBeast Burgers, ORG)",ORG,MrBeast Burgers,True
1773,2020-05-08T17:16:17.000Z,1258807951164239873,0.856750,283,205,A 2020 peer-reviewed study published in the jo...,299,1,NaN,103443,"(US, GPE)",GPE,US,True
1773,2020-05-08T17:16:17.000Z,1258807951164239873,0.856750,283,205,A 2020 peer-reviewed study published in the jo...,299,1,NaN,103443,"(DoD, ORG)",ORG,DoD,True


In [ ]:
claim_propagation[claim_propagation['target_filter']==True].to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation_NER_explode.h5',compression='xz')
claim_propagation[claim_propagation['target_filter']==True]

,tweet_user,tweet_id,like_count,depth,parent_user,create_date,parent_id,retweet_count,post_text,claim_number,group,hashtag,NER,target_type,target,target_filter
0,64,1260590991641362436,301,0,0,2020-05-13T15:21:27.000Z,0,113,So you left a 69 page guidebook but no PPE or ...,141,1,NaN,"(PPE, ORG)",ORG,PPE,True
1,135,1288771853822103552,194,0,0,2020-07-30T09:42:08.000Z,0,99,More tales from the NHS frontline are emerging...,138,0,NaN,"(NHS, ORG)",ORG,NHS,True
1,135,1288771853822103552,194,0,0,2020-07-30T09:42:08.000Z,0,99,More tales from the NHS frontline are emerging...,138,0,NaN,"(wards &amp, ORG)",ORG,wards &amp,True
2,290,1251209120528637952,7650,0,0,2020-04-17T18:01:15.000Z,0,4279,DEVELOPING: A crowd of protesters could be see...,30,0,NaN,"(@GovTimWalz, PERSON)",PERSON,@GovTimWalz,True
2,290,1251209120528637952,7650,0,0,2020-04-17T18:01:15.000Z,0,4279,DEVELOPING: A crowd of protesters could be see...,30,0,NaN,"(Minnesotans, ORG)",ORG,Minnesotans,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221248,144736,1235937681047080960,0,1,106463,2020-03-06T14:38:00.000Z,1235932698688053249,0,"#WaterIsLife \nremember your vote matters, \nC...",229,2,WaterIsLife#WhenWeAllVote#FlintWaterCrisis,NaN,,[],True
221250,144738,1242264347025571842,0,1,93248,2020-03-24T01:37:55.000Z,1242257594540056576,1,NO @GOP should even entertain this anti-Americ...,96,0,NaN,"(@realDonaldTrump @senatemajldr, ORG)",ORG,@realDonaldTrump @senatemajldr,True
221251,144739,1234597235746394118,0,1,56172,2020-03-02T21:51:33.000Z,1234592892896923652,0,I assume if you win it will the begging of the...,173,3,NaN,"(AMERICAN IMPERIALISM, ORG)",ORG,AMERICAN IMPERIALISM,True
221251,144739,1234597235746394118,0,1,56172,2020-03-02T21:51:33.000Z,1234592892896923652,0,I assume if you win it will the begging of the...,173,3,NaN,"(https://t.co/JFI65K1n1U, ORG)",ORG,https://t.co/JFI65K1n1U,True
